# Chapter 4 : cryptocipher

4.21 : given a message, create a dictionnary mapping the letters in message to the alphabet

In [259]:
import string
import random
import numpy as np
import requests
import re
from operator import itemgetter

# Create random cipher

In [260]:
alphabet = list(string.ascii_lowercase)
shuffled_alphabet = list(string.ascii_lowercase)
random.shuffle(shuffled_alphabet)
cipher = {x:y for x,y in zip(alphabet, shuffled_alphabet)}

In [261]:
cipher

{'a': 'r',
 'b': 'v',
 'c': 'u',
 'd': 'f',
 'e': 'g',
 'f': 'c',
 'g': 'h',
 'h': 'n',
 'i': 'o',
 'j': 'q',
 'k': 't',
 'l': 'b',
 'm': 'm',
 'n': 'i',
 'o': 'j',
 'p': 'k',
 'q': 'd',
 'r': 'w',
 's': 'e',
 't': 'x',
 'u': 's',
 'v': 'p',
 'w': 'z',
 'x': 'y',
 'y': 'l',
 'z': 'a'}

# Train language model

Create a character-level Markov model based on an English dataset (an edit of https://www.gutenberg.org/ebooks/2701 ). Any book could be used instead or together with this one. We suppose that the probability $p(a_k|a_1, ..., a_{k-1})=p(a_k|a_{k-1})$. In other terms, it only depend on the previous character. We will count occurances of character pairs in the text and will divide it by the count of character occurances in the text. 

For a given word : 
logprob$(word) = \log (p(x_1)  \Pi_{i=2}^n p(x_t | x_{t-1}))$

Markov matrix will provide the counts of pairs $a_i \rightarrow a_j$.
Weights vector will provide the counts of each letter.

In [262]:
markov_matrix = np.ones((26,26))
weights = np.zeros(26)

In [263]:
def markov_update(a, b):
    markov_matrix[alphabet.index(a), alphabet.index(b)]+=1

def weight_update (a):
    weights[alphabet.index(a)]+=1

### Get the log-probability of a word

In [264]:

def get_word_prob(word : str):

    i = alphabet.index(word[0])
    logp = np.log(weights[i])

    for ch in word[1:]:
        j = alphabet.index(ch)
        logp += np.log(markov_matrix[i, j]) 
        i = j
    return logp

### Get the probability of a sentence

The sentence is stripped from eventual punctuation and transformed to lower case before calculation of probability.


In [265]:
def get_sequence_prob(words:str):
    words = words.split()
    logp = sum([get_word_prob(word) for word in words])
    return logp

### Get a reference file for language model training.

We will use "Moby Dick" by Herman Melville.
Source : https://www.gutenberg.org/ebooks/2701

In [266]:
if not os.path.exists('moby_dick.txt'):
    print("Downloading moby dick...")
    r = requests.get('https://www.gutenberg.org/files/2701/2701-0.txt')
    with open('moby_dick.txt', 'w') as f:
        f.write(r.content.decode())
    with open('moby_dick.txt', 'r') as f:
        r = f.readlines()[848:21965]
    with open('moby_dick.txt', 'w') as f:
        f.write('\n'.join(r))

### Train the model

In [267]:
regex = re.compile('[^a-zA-Z]')

for line in open('moby_dick.txt'):
  line = line.strip()

  if line!='':
    line = regex.sub(' ', line)
    tokens = line.lower().split()

    for token in tokens:
      t0 = token[0]
      weight_update(t0)

      for t1 in token[1:]:
        markov_update(t0, t1)
        t0 = t1

### Normalize the probabilities

In [268]:
weights /= weights.sum()
markov_matrix /= markov_matrix.sum(axis=1, keepdims=True)

### Read and encode the message

In [269]:
with open('encode_message.txt', 'r') as f:
    message = f.read().replace('\n',' ').lower()
    regex = re.compile('[^a-zA-Z]')
    message_to_encode = ' '.join(regex.sub(' ', message).strip().split())
message_to_encode

'i then lounged down the street and found as i expected that there was a mews in a lane which runs down by one wall of the garden i lent the ostlers a hand in rubbing down their horses and received in exchange twopence a glass of half and half two fills of shag tobacco and as much information as i could desire about miss adler to say nothing of half a dozen other people in the neighbourhood in whom i was not in the least interested but whose biographies i was compelled to listen to away they went and i was just wondering whether i should not do well to follow them when up the lane came a neat little landau the coachman with his coat only half buttoned and his tie under his ear while all the tags of his harness were sticking out of the buckles it hadn t pulled up before she shot out of the hall door and into it i only caught a glimpse of her at the moment but she was a lovely woman with a face that a man might die for my cabby drove fast i don t think i ever drove faster but the others 

In [270]:
def encode(message : str, cipher : dict):
    encoded = ''
    for c in message:
        if c==' ':
            encoded += c
        else:
            encoded += cipher[c]
    return encoded

def decode(message : str, cipher : dict):
    anticipher = {x: y for y, x in cihper.items()}
    decoded = ''
    for c in message:
        if c==' ':
            decoded += c
        else:
            encoded += anticipher[c]
    return decoded

In [271]:
code = encode(message_to_encode, cipher)
code

'o xngi bjsihgf fjzi xng exwggx rif cjsif re o gykguxgf xnrx xngwg zre r mgze oi r brig znoun wsie fjzi vl jig zrbb jc xng hrwfgi o bgix xng jexbgwe r nrif oi wsvvoih fjzi xngow njwege rif wgugopgf oi gyunrihg xzjkgiug r hbree jc nrbc rif nrbc xzj cobbe jc enrh xjvruuj rif re msun oicjwmrxoji re o ujsbf fgeowg rvjsx moee rfbgw xj erl ijxnoih jc nrbc r fjagi jxngw kgjkbg oi xng igohnvjswnjjf oi znjm o zre ijx oi xng bgrex oixgwgexgf vsx znjeg vojhwrknoge o zre ujmkgbbgf xj boexgi xj rzrl xngl zgix rif o zre qsex zjifgwoih zngxngw o enjsbf ijx fj zgbb xj cjbbjz xngm zngi sk xng brig urmg r igrx boxxbg brifrs xng ujrunmri zoxn noe ujrx jibl nrbc vsxxjigf rif noe xog sifgw noe grw znobg rbb xng xrhe jc noe nrwigee zgwg exoutoih jsx jc xng vsutbge ox nrfi x ksbbgf sk vgcjwg eng enjx jsx jc xng nrbb fjjw rif oixj ox o jibl urshnx r hbomkeg jc ngw rx xng mjmgix vsx eng zre r bjpgbl zjmri zoxn r crug xnrx r mri mohnx fog cjw ml urvvl fwjpg crex o fji x xnoit o gpgw fwjpg crexgw vsx xng jxngwe 

# Use a genetic algorithm to decript the message

### parameters

In [272]:
N = 32
keep = 8
children = 3
epocs = 300

In [273]:
def generate_dna():
    alpha = list(string.ascii_lowercase)
    random.shuffle(alpha)
    return ''.join(alpha)

In [274]:
dna_pool = [generate_dna() for _ in range(N)]

In [275]:
def evaluate_dna(dna, message):
    alpha = list(string.ascii_lowercase)
    encoder = {x:y for x, y in zip(alpha, list(dna))}
    dna_code = encode(message, encoder)
    score = get_sequence_prob(dna_code)
    return score


In [276]:
def keep_best(pool, n=keep):
    score = [(dna, evaluate_dna(dna, code)) for dna in pool]
    score = sorted(score, key=itemgetter(1), reverse=True)[:n]
    best_dna = [x[0] for x in score]
    best_score = score[0][1]
    return best_dna, best_score

In [277]:
def mutations(pool, nc=children):
    children = []
    for dna in pool:
        for _ in range(nc):
            child = list(dna)
            i=random.randint(0,25)
            j=random.randint(0,25)
            child[i], child[j] = child[j], child[i]
            child = ''.join(child)
            children.append(child)
    return pool+children


In [278]:
len(dna_pool[0])

26

In [279]:
for i in range(epocs):
    dna_pool, best_score = keep_best(dna_pool)
    dna_pool = mutations(dna_pool)
    if i % 10 == 0:
        print(i, 'iterations, best score : ', best_score)





0 iterations, best score :  -5773.357436127759
10 iterations, best score :  -4688.024226029333
20 iterations, best score :  -4109.492605273266
30 iterations, best score :  -3928.3362549601097
40 iterations, best score :  -3775.449636702807
50 iterations, best score :  -3677.969221579938
60 iterations, best score :  -3662.294338695646
70 iterations, best score :  -3559.3692414654033
80 iterations, best score :  -3523.554328999374
90 iterations, best score :  -3506.700081724467
100 iterations, best score :  -3448.42367456203
110 iterations, best score :  -3425.568403827853
120 iterations, best score :  -3412.0399081432847
130 iterations, best score :  -3330.347765747104
140 iterations, best score :  -3318.3239665983015
150 iterations, best score :  -3317.877877775549
160 iterations, best score :  -3317.097394636357
170 iterations, best score :  -3278.195529057987
180 iterations, best score :  -3147.0147003876746
190 iterations, best score :  -2874.9333587264764
200 iterations, best score

In [281]:
 best_dna = dna_pool[0]
 alpha = list(string.ascii_lowercase)
 encoder = {x:y for x, y in zip(alpha, list(best_dna))}
 encode(code, encoder)

'i then lounged down the street and found as i expected that there was a mews in a lane which runs down by one wall of the garden i lent the ostlers a hand in rubbing down their horses and received in exchange twopence a glass of half and half two fills of shag tobacco and as much information as i could desire about miss adler to say nothing of half a dozen other people in the neighbourhood in whom i was not in the least interested but whose biographies i was compelled to listen to away they went and i was qust wondering whether i should not do well to follow them when up the lane came a neat little landau the coachman with his coat only half buttoned and his tie under his ear while all the tags of his harness were sticking out of the buckles it hadn t pulled up before she shot out of the hall door and into it i only caught a glimpse of her at the moment but she was a lovely woman with a face that a man might die for my cabby drove fast i don t think i ever drove faster but the others 